In [2]:
import torch
import torchvision
import torchvision.transforms as tr # 데이터불러오면서 전처리까지 할 수 있음
from torch.utils.data import DataLoader, Dataset # 배치 사이즈 만들어줌 / 데이터 튜닝
import numpy as np

### 1. 파이토치 제공 데이터 사용

In [3]:
transf = tr.Compose([tr.Resize(8),tr.ToTensor()])
# 전처리 방법 'transf' 대해서 정의하는거임 : 8*8 resize 후에 Tensor로 바꿔주겠다는 것
# 하지만 이렇게 하기위해선 입력받은 이미지가 PIL 이여야 해서 임의로 정의해줘야 하는 경우가 많음

내장 데이터 다운받기

In [5]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
# ./data 위치에 CIFAR10 의 train 데이터를 transf 전처리 방법으로 다운로드 받겠다는 것
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transf)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
trainset[0][0].size() # 기존과 다르게 channel 수가 제일 앞에 나옴 

torch.Size([3, 8, 8])

In [7]:
# batch size 50개, sub process 2개로 진행한다
trainloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=50, shuffle=True, num_workers=2)
len(trainloader) # trainset이 총 50,000개인데 50으로 나누어 총 1,000개가 있음! 

1000

In [8]:
detaiter = iter(trainloader)
images, labels = detaiter.next()
images.size() # batch, channel, size 순서!

torch.Size([50, 3, 8, 8])

**-> tensor와 ndarray는 순서가 다름!!**
- tensor는 channel이 batch 다음임
- ndarray는 channel이 제일 마지막

### 2. 개인 데이터 사용하기 - 1) 같은 클래스 별 폴더 이미지 데이터 사용

- 클래스 별 다르게 저장되어 있다고 가정 : ./class/tiger & ./class/lion 이런식

In [11]:
'''
transf = tr.Compose([tr.Resize(16), tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='./class', transform=transf)
trainloader = DataLoader(trainset, batch_size=10, shuffle=False, num_workers=2)
-> 이렇게만으로 데이터도 불러오고 레이블 자동으로 매겨지고 전처리까지 되는거임!
'''

"\ntransf = tr.Compose([tr.Resize(16), tr.ToTensor()])\ntrainset = torchvision.datasets.ImageFolder(root='./class', transform=transf)\ntrainloader = DataLoader(trainset, batch_size=10, shuffle=False, num_workers=2)\n-> 이렇게만으로 데이터도 불러오고 레이블 자동으로 매겨지고 전처리까지 되는거임!\n"

### 3. 개인 데이터 사용 - 2)

방법 1.
- 위의 방법의 경우 디렉토리를 함부러 건드리기 애매한경우나 폴더 단위가 아닌 sql로 받아오는 경우 등에는 사용불가
- pytorch의 transform이 제한적이라 디테일은 직접 작성해줘야할 필요가 있음

In [20]:
train_images = np.random.randint(256, size=(20,32,32,3)) # ndarray는 32*32 사이즈 3 channel 20개
train_labels = np.random.randint(2, size=(20,1)) # 그에 따른 label도 20개
'''
그리고 class 객체 만들기 전에 여기서 preprocessing 과정을 거침
train_images, train_labels = preprocessing(train_images, train_labels) 이런식으로
'''
print(train_images.shape, '\t', train_labels.shape)

(20, 32, 32, 3) 	 (20, 1)


In [33]:
'''
* 정형화됨 *
'Dataset'을 상속받으면서
'__init__', '__getiem__', '__len_'
을 갖는 class를 하나 만든다
'''

class TensorData(Dataset) :
    
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data) # x_data를 float유형의 tensor로 변경함
        self.x_data = self.x_data.permute(0,3,1,2) # 기존 순서에서 순서를 변경해주는거임 : 총 개수, 채널 수, width, height
        self.y_data = torch.LongTensor(y_data) # y_data도 long 유형의 tensor로 변경
        self.len = self.y_data.shape[0]
        
    def __getitem__(self, index) :
        sample = self.x_data[index], self.y_data[index] # 그 index에 해당하는 값을 튜플로 반환하는 함수
        return sample
    
    def __len__(self):
        return self.len

In [ ]:
# 1. class 객체를 만들어준다
train_data = TensorData(train_images, train_labels)
# 2. 이를 DataLoader에 통과시켜서 batch 단위로 만들어준다
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

In [23]:
train_data[0][0].size() # 순서가 변경됨!

torch.Size([3, 32, 32])

방법2.
- transform 기능 더하기! => transform에 대한 필요한 class 들을 다 작성하는 것임!

In [35]:
# 0. tensor로 바꾸는 기능을 별도의 class 함수로 빼고 transform 할지 여부를 init 함수에 포함한다
# 1. transofrm을 한다고하면 그냥 튜플 내보내기전에 transform을 하고 튜플을 return 한다
# 2. 이 튜플을 ToTensor로 받아서 tensor 형태로 바꾸어 준다

class MyDataset(Dataset):
    
    def __init__(self, x_data, y_data, transform = None):
        '''
        self.x_data = torch.FloatTensor(x_data) 
        self.x_data = self.x_data.permute(0,3,1,2)
        self.y_data = torch.LongTensor(y_data) 
        '''
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.len = self.y_data.shape[0]
        
    def __getitem__(self, index) :
        sample = self.x_data[index], self.y_data[index] # 그 index에 해당하는 값을 튜플로 반환하는 함수
        
        if self.transform :
            sample = self.transform(sample)
            
        return sample
    
    def __len__(self):
        return self.len
    
class ToTensor_customed:
    
    def __call__(self, sample):
        inputs, labels = sample # sample 이 튜플 형태니까 이렇게 받을 수 있음
        # 여기에서 tensor로 바꾸고 순서 바꾸는 작업 수행
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)
        labels = torch.LongTensor(labels)
        return inputs, labels
    
# 그리고 지금 할 연산 관련 class까지
class LinearTensor :
    
    def __init__(self, weight=1, bias=0):
        self.weight = weight
        self.bias = bias
        
    def __call__(self, sample):
        inputs, labels = sample
        inputs = self.weight*inputs + self.bias
        return inputs, labels

In [36]:
trans = tr.Compose([ToTensor_customed(), LinearTensor(2,5)]) # tensor 바꾸기 위해 trans 정의할때 여기서 사용

train_images = np.random.randint(256, size=(20,32,32,3)) # ndarray는 32*32 사이즈 3 channel 20개
train_labels = np.random.randint(2, size=(20,1)) # 그에 따른 label도 20개

ds1 = MyDataset(train_images, train_labels, transform=trans)

train_loader1 = DataLoader(ds1, batch_size=10, shuffle=True)

In [37]:
first_data = ds1[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [39]:
dataiter1= iter(train_loader1)
images1, labels1 = dataiter1.next()
images1 # r값 분포가 0 ~ 255 여야 하는데 넘어가는 걸 보니 정상적으로 된 것을 알 수있음

tensor([[[[165., 107., 345.,  ...,  75., 459., 395.],
          [481., 161., 117.,  ..., 107., 421., 231.],
          [121., 197., 227.,  ..., 301., 335., 185.],
          ...,
          [387., 177.,  47.,  ..., 155., 339., 315.],
          [447., 433.,  37.,  ..., 391., 383., 233.],
          [389., 355., 317.,  ..., 293., 107., 457.]],

         [[309., 157., 323.,  ...,  61., 453., 267.],
          [207., 201., 253.,  ..., 489., 321., 167.],
          [275.,  21., 463.,  ...,  87., 317., 215.],
          ...,
          [ 45., 147.,  57.,  ..., 437.,  13., 323.],
          [  5., 105., 195.,  ..., 355., 319., 169.],
          [363., 425., 211.,  ...,  79., 315., 391.]],

         [[399., 217., 267.,  ..., 411.,  61., 471.],
          [ 61., 255., 157.,  ...,  57., 257., 513.],
          [ 57., 321., 409.,  ..., 105., 453., 157.],
          ...,
          [ 21., 235., 365.,  ...,  91.,  19.,  15.],
          [265., 257., 387.,  ...,  29., 451., 257.],
          [101., 187.,  49.,  ...